# QC, analysis of Gutenkunst three pop out of Africa
Here, we would like to do a sanity check that our models are producing similar results to that found 
in Gutenkunst 2009.  https://doi.org/10.1371/journal.pgen.1000695


In [ ]:
import msprime
from stdpopsim import homo_sapiens
import allel
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from allel.util import asarray_ndim, check_integer_dtype

In [ ]:
ch = "chr22"
chrom = homo_sapiens.genome.chromosomes[ch]
model = homo_sapiens.GutenkunstThreePopOutOfAfrica()

pops = ["YRI","CEU","CHB"]
treeSequencePath = "./HomoSapians3pop_"+ch+"_50samplesEach.trees"

The tree sequence for this simulation exists at the path 'treeSequencePath',
there is no need to re-simulate if it still exists.
The simulation only takes about a minute for a single chromosome.

In [ ]:
# 50 samples each from YRI, CEU and CHB.
samples = [msprime.Sample(population=j, time=0) for j in range(3) for _ in range(50)]

ts = msprime.simulate(
    samples=samples,
    recombination_map=chrom.recombination_map(),
    mutation_rate=chrom.mean_mutation_rate,
    **model.asdict())
ts.dump(treeSequencePath)

First, lets take a look at comparisons to nucleotide diversity within all three populations.

In [ ]:
ts = msprime.load(treeSequencePath)
gm = ts.genotype_matrix()

#make a haplotype data struct out of each population gentype matrices
haplotype_arrays = [allel.HaplotypeArray(gm[:,ts.samples(population=i)]) for i in range(3)]
total_haplo = allel.HaplotypeArray(gm)

#counts all ancestral/derived alleles at each site
allele_counts = [pop.count_alleles() for pop in haplotype_arrays]
total_ac = total_haplo.count_alleles()

#positions of all SNPs
pos = np.array([s.position for s in ts.sites()],dtype='float32')

for i,ac in enumerate(allele_counts):
    nd = allel.sequence_diversity(pos=pos,ac=ac)
    print("pop %s nucleotide diversity = " %(pops[i]),nd)
    
td = allel.sequence_diversity(pos=pos,ac=total_ac)
print("\ntotal nucleotide diversity: ",td)